In [3]:
import itertools
import northstar
import numpy as np
import pandas as pd
import scanpy as sc

## Assigning cell type by nearest neighbor in an atlas of cells

In [6]:
def assign_celltype(adata, atlas, key="new_celltype", run="chunk"):
    ##Assign closest celltype from atlas
    if run == "chunk":
        new_celltype = []
        for step in range(0, len(adata.obs_names), 1000):
            index_chunk = adata.obs_names.tolist()[step : step + 1000]
            chunk_data = adata[index_chunk, :]
            model = northstar.Subsample(
                atlas=atlas,
                n_features_per_cell_type=100,
                n_features_overdispersed=0,
                features_additional=None,
                n_pcs=50,
                n_neighbors=0,
                n_neighbors_external=20,
                distance_metric="correlation",
                threshold_neighborhood=0.5,
                clustering_metric="cpm",
                resolution_parameter=0.0001,
                normalize_counts=True,
                join="keep_first",
            )
            model.fit(chunk_data)
            cell_types_newdata = model.membership
            new_celltype = new_celltype + [str(x) for x in cell_types_newdata]
            del model
        adata.obs[key] = new_celltype
    elif run == "all":
        model = northstar.Subsample(
            atlas=atlas,
            n_features_per_cell_type=100,
            n_features_overdispersed=0,
            features_additional=None,
            n_pcs=50,
            n_neighbors=0,
            n_neighbors_external=20,
            distance_metric="correlation",
            threshold_neighborhood=0.5,
            clustering_metric="cpm",
            resolution_parameter=0.0001,
            normalize_counts=True,
            join="keep_first",
        )
        model.fit(adata)
        cell_types_newdata = model.membership
        adata.obs[key] = [str(x) for x in cell_types_newdata]
    else:
        print(f'Run must equal one of ["chunk","all"]')
    return

## Create dictionary of stats of interest for every combination of obs columnd in anndata object

In [5]:
def stats_dict(adata, obs_list):
    """Creates dictionary of all stats for every combination of metadata in obs_list
    
    cell type category has to be first in obs list for this to work
    """
    condensed_dictionary = {}
    num_obs = len(obs_list) + 1
    for ind in range(0, num_obs):
        for subset in itertools.combinations(obs_list, ind):
            if len(subset) != 0:
                subset = list(subset)
                key = "_".join(subset)
                if obs_list[0] not in subset: #
                    continue
                print(subset)
                if len(subset) == 1:
                    adata.obs[key] = adata.obs[subset].values
                else:
                    adata.obs[key] = ["_".join(x) for x in adata.obs[subset].values]
                run_stats(adata, condensed_dictionary, key)
    return condensed_dictionary

## Collect stats for each obs column and return to dictionary
### cell count
### gene expresion matrix
### proportion expressing matrix

In [5]:
def run_stats(adata, dct, key, mean="all"):
    """internal function of condense_adata functions"""
    index = sorted(adata.obs[key].unique())
    ## counts, averages and proportions for each obs/var column in list
    dct[key] = {}
    obs_value_count = pd.DataFrame(adata.obs[key].value_counts()).T
    dct[key]["cell_count"] = obs_value_count
    if mean == "all":
        gene_expr_mtx = pd.DataFrame(columns=adata.var_names, index=index, dtype = np.float16)
        gene_perecent_expr_mtx = pd.DataFrame(columns=adata.var_names, index=index, dtype = np.float16)
        for group in index:
            group_adata = adata[adata.obs[key].isin([group]), :].copy()
            gene_expr_mtx.loc[group] = group_adata.X.mean(0)
            gene_perecent_expr_mtx.loc[group] = np.array(
                np.squeeze((group_adata.X > 0).sum(axis=0)) / len(group_adata.obs_names)
            )
        dct[key]["gene_expression_average"] = gene_expr_mtx
        dct[key]["gene_proportion_expression"] = gene_perecent_expr_mtx
    return

## Write condensed stats dict to .h5 file

In [6]:
def condense_adata_metadata(adata, obs_list, output):
    condensed_dictionary = stats_dict(adata, obs_list)
    hdf = pd.HDFStore(output)
    for key in condensed_dictionary.keys():
        for key2 in condensed_dictionary[key].keys():
            hdf.put(f"{key}/{key2}", condensed_dictionary[key][key2])
    hdf.close()
    return

## load datasets and do some final cleaning to align

In [7]:
# data handling for alignment
acz_adata = sc.read(
    "/home/carsten/alvira_bioinformatics/data/alvira_sc_data/processed/201013_allcells_from_goodloom_cpm.gz.h5ad"
)
sc.pp.normalize_total(acz_adata, target_sum=1e6, key_added="coverage")
acz_adata.obs["CellType"] = acz_adata.obs["cellSubtype"]
acz_adata.obs["NumberOfCells"] = [
    acz_adata.obs["CellType"].value_counts()[x] for x in acz_adata.obs["CellType"]
]
acz_adata.obs["dataset"] = "ACZ"
acz_adata = acz_adata[acz_adata.obs["Treatment"] == "normal", :]
acz_adata.X = acz_adata.X.todense()
tms_adata = sc.read(
    "/home/carsten/alvira_bioinformatics/data/external_datasets/ingested/tabulamurissenis2020_facs.gz.h5ad"
)
tms_adata.obs["dataset"] = "TMS"
hurs_adata = sc.read(
    "/home/carsten/alvira_bioinformatics/data/external_datasets/ingested/hurskainen2021.gz.h5ad"
)
hurs_adata.obs["dataset"] = "Hurskainen2021"
hurs_adata = hurs_adata[hurs_adata.obs["Treatment"] == "Normoxia", :]

## Create condensed atlas and save combined h5ad

In [8]:
# create condensed atlas
atlas = northstar.subsample_atlas(acz_adata, n_cells=50)
obs_list = [
    "dataset",
    "new_celltype",
    "Timepoint",
]
new_datasets = [tms_adata, hurs_adata]
for adata in new_datasets:
    assign_celltype(adata, atlas)
    adata.obs["celltype"] = adata.obs["new_celltype"]
acz_adata.obs["celltype"] = acz_adata.obs["cellSubtype"]
final_adata = acz_adata.concatenate(new_datasets, join="outer", fill_value=0)
final_adata.obs["timepoint"] = final_adata.obs["Timepoint"]

condense_adata_metadata(
    final_adata,
    ["celltype", "dataset", "timepoint"],
    output="/home/carsten/alvira_bioinformatics/condensed_atlas/data/condensed_lung_atlas.h5",
)
del final_adata.var
final_adata.write(
    "/home/carsten/alvira_bioinformatics/condensed_atlas/data/lung_atlas.gz.h5ad",
    compression="gzip",
)

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


['celltype']
['celltype', 'dataset']
['celltype', 'timepoint']
['celltype', 'dataset', 'timepoint']


/home/carsten/anaconda3/envs/acz_sc/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Mousename' as categorical
/home/carsten/anaconda3/envs/acz_sc/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'Timepoint' as categorical
/home/carsten/anaconda3/envs/acz_sc/lib/python3.7/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and w